# Projet maison n° 4

In [1]:
# imports
import pandas as pd

## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

2. Implémenter une fonction Python qui produit un unique DataFrame avec tous les fichiers en utilisant pandas  (par ex. fonction \"concat\" ou méthode \"append\"), pas de bash :)

Ordre et noms des colonnes : 'year', 'name', 'gender', 'births'

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini par les différents fichiers (voir la documentation ci-dessus).

In [2]:
# 1) implémentation avec glob + append
import glob

def df_names_us():
    df = pd.DataFrame()
    files = glob.glob('names/*.txt')
    files.sort()
    for filename in files:
        csv = pd.read_csv(filename,
                          names=['name', 'gender', 'births'])
        csv['year'] = int(filename[-8:-4]) # yobAAAA.txt
        df = df.append(csv, ignore_index=True)
    df = df[['year', 'name', 'gender', 'births']]
    return df

La fonction glob renvoie une liste, potentiellement vide, de chemins correspondant au motif pathname, qui doit être une chaîne de caractères contenant la spécification du chemin. 
https://docs.python.org/fr/3.6/library/glob.html

In [3]:
%%time
df_us = df_names_us()
df_us.shape

CPU times: user 3.78 s, sys: 1.19 s, total: 4.97 s
Wall time: 4.97 s


(1989401, 4)

In [4]:
# head
df_us.head()

,year,name,gender,births
0,1880,Mary,F,7065
1,1880,Anna,F,2604
2,1880,Emma,F,2003
3,1880,Elizabeth,F,1939
4,1880,Minnie,F,1746


Concaténation des différents dataframe

In [5]:
# 2) implémentation avec range + concat
def df_names_us():
    dfs = []
    for year in range(1880, 2020):
        csv = pd.read_csv(f'names/yob{year}.txt',
                          names=['name', 'gender', 'births'])
        csv['year'] = year
        dfs.append(csv)
    df = pd.concat(dfs, ignore_index=True)
    df = df[['year', 'name', 'gender', 'births']]
    return df

In [6]:
%%time
df_us = df_names_us()
df_us.shape

CPU times: user 1.21 s, sys: 193 ms, total: 1.41 s
Wall time: 1.41 s


(1989401, 4)

In [7]:
# head
df_us.head()

,year,name,gender,births
0,1880,Mary,F,7065
1,1880,Anna,F,2604
2,1880,Emma,F,2003
3,1880,Elizabeth,F,1939
4,1880,Minnie,F,1746


**slice vs regex**

In [8]:
# slice 140 x 96.1ns = ~14ms
%timeit 'yob2020.txt'[-8: -4]

96.9 ns ± 2.1 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [9]:
# regex 140 x 1.37µs = ~192ms (x14)
import re
%timeit re.search('(\d+)', 'yob2020.txt').group(0)

937 ns ± 48.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2019_csv.zip


Lire la documentation, ça peut être utile...
 
2. Implémenter une fonction Python qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : year, name, gender, births
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne 'gender'
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : années (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [10]:
def df_names_fr():
    # genres
    d = {'1': 'M', '2': 'F'}
    # read_csv
    df = pd.read_csv('nat2019.csv',
                      sep=';',
                      header=0,
                      names=['gender', 'name', 'year', 'births'],
                      converters={
                          'gender': d.get,
                          'name': str.title
                      })
    # clean
    df = df.loc[(df['name'].str.len() > 1)
                & (df['year'] != 'XXXX')
                & ~df['name'].str.startswith('_')]
    # types
    df['year'] = df['year'].astype(int)
    # ordre colonnes
    df = df[['year', 'name', 'gender', 'births']]
    # tri
    df = df.sort_values(['year', 'gender', 'births', 'name'],
                   ascending=[True, True, False, True])
    df = df.reset_index(drop=True)
    
    return df

In [11]:
%%time
df_fr = df_names_fr()
df_fr.shape

CPU times: user 1.19 s, sys: 86.5 ms, total: 1.28 s
Wall time: 1.28 s


(615912, 4)

In [12]:
# head
df_fr.head()

,year,name,gender,births
0,1900,Marie,F,48713
1,1900,Jeanne,F,13981
2,1900,Marguerite,F,8058
3,1900,Germaine,F,6981
4,1900,Louise,F,6696


In [13]:
# prénom NA
df_fr.loc[df_fr['name']=='Na']

,year,name,gender,births
404212,2003,Na,F,3


In [14]:
# impact du converters / name
d = {'1': 'M', '2': 'F'}

df = pd.read_csv('nat2019.csv',
                  sep=';',
                  header=0,
                  names=['gender', 'name', 'year', 'births'],
                  converters={
                      'gender': d.get,
                      #'name': str.title
                  })

df['name'].isnull().sum()

2

converters : dict. optional
    Dict of functions for converting values in certain columns. Keys can either
    be integers or column labels

In [15]:
# capitalize
'JEAN-MARIE'.capitalize()

'Jean-marie'

In [16]:
# title
'JEAN-MARIE'.title()

'Jean-Marie'

In [17]:
# comment faire title avec capitalize :)
'-'.join(map(str.capitalize, 'JEAN-MARIE'.split('-')))

'Jean-Marie'

## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/browseBox.do?node=5385566

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3.
 
1. Télécharger le dataset des taux de change : http://webstat.banque-france.fr/fr/downloadFile.do?id=5385698&exportType=csv


2. Implémenter une fonction qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction pd.to_datetime() avec le format '%d/%m/%Y'). Les colonnes du DataFrame doivent correspondre aux devises.

In [18]:
def df_taux_change(devises):
    df = pd.read_csv("Webstat_Export.csv",
                     sep=";",
                     na_values='-',
                     decimal=',',
                     skiprows=[0, 1, 3, 4, 5],  # le skiprows permet à l'option "decimal" de fonctionner
                     converters={0: lambda x: pd.to_datetime(x, format='%d/%m/%Y', errors='ignore')})

    # extraction des codes monnaies
    cols = pd.Series(df.columns.tolist()).str.extract('\(([A-Z]{3})\)', expand=True) ## ici je veux seulement 3 caractères
    cols.iloc[0] = 'Date'
    df.columns = cols[0]

    # selection des devises
    df = df[['Date'] + devises]

    # drop na
    df = df.dropna()

    # set index
    df = df.set_index('Date')
    
    return df

Les fonctions Decimal et Thousand ne marchent que si la colonne est bien formée i.e. que les chiffres sont de la même forme. Le skiprows permet d'avoir des données pures. 

In [19]:
%%time
df_tx = df_taux_change(['CHF', 'GBP', 'USD'])
df_tx.shape

CPU times: user 946 ms, sys: 12.7 ms, total: 959 ms
Wall time: 960 ms


(5591, 3)

In [20]:
# head
df_tx

,CHF,GBP,USD
Date,,,
2020-11-03,1.0709,0.90042,1.1702
2020-11-02,1.0695,0.90053,1.1652
2020-10-30,1.0698,0.90208,1.1698
2020-10-29,1.0684,0.90430,1.1704
2020-10-28,1.0693,0.90662,1.1727
...,...,...,...
1999-01-08,1.6138,0.70940,1.1659
1999-01-07,1.6165,0.70585,1.1632
1999-01-06,1.6116,0.70760,1.1743


### Tests

In [21]:
import unittest

class Lesson4Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 1989401)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 615912)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test names
        self.assertTrue(df.loc[df['name'].str.contains('^[A-Z]+(?:-[A-Z]+)?$')].empty)
        # test gender
        self.assertEqual(len(df), len(df.loc[df['gender']=='F']) + len(df.loc[df['gender']=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

    def test_df_taux_change(self):
        df = df_taux_change(['CHF', 'GBP', 'USD'])
        # colonnes
        self.assertEqual(list(df.columns), ['CHF', 'GBP', 'USD'])
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.datetimes.DatetimeIndex))
        # types taux
        self.assertTrue((df.dtypes == 'float').all())
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

In [22]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Lesson4Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [23]:
# run tests

run_tests()

test_df_names_fr (__main__.Lesson4Tests) ... ok
test_df_names_us (__main__.Lesson4Tests) ... ok
test_df_taux_change (__main__.Lesson4Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 5.116s

OK


### Un peu de Data Science...

**Question n° 1**

Pourquoi le value_counts() du "gender" donne-t-il un tel écart entre F et M ?

In [24]:
# value_counts du gender

df_us['gender'].value_counts()

F    1174655
M     814746
Name: gender, dtype: int64

In [25]:
df_fr['gender'].value_counts()

F    335205
M    280707
Name: gender, dtype: int64

In [26]:
# pct_change

df_us['gender'].value_counts().pct_change()

F         NaN
M   -0.306395
Name: gender, dtype: float64

pct_change : Donne le pourcentage de différence.pourcentage d'une ligne par rapport à la ligne n-1

Pas assez de diversité dans les prénoms des M. 
Il y a une ligne par prénoms, années  et genre => il y a plus de lignes avec "F"....

**Question n° 2**

Pourquoi le value_counts() du "name" donne-t-il ce résultat ?

In [27]:
# value_counts du gender

df_us['name'].value_counts().head(16)

Lee        280
John       280
Tommie     280
William    280
Ollie      280
Jesse      280
James      280
Marion     280
Jessie     280
Francis    280
Johnnie    280
Leslie     280
Jean       280
Sidney     280
Joseph     280
Charles    279
Name: name, dtype: int64

In [28]:
# value_counts du gender

df_fr['name'].value_counts().head(16)

Ange         240
Alix         240
Camille      240
Dominique    237
Claude       233
Hyacinthe    229
Maxime       228
Marie        212
Gaby         208
Cyrille      207
Elie         200
Andréa       199
France       198
Irène        195
Léandre      195
Stéphane     193
Name: name, dtype: int64

Les prénoms sont donnés par année. On a toujours 2 lignes chaque année pour les prénoms US dont le compte est 280 et pour les prénoms français est 240. Les prénoms ont été données au deux genres. 

**Question n° 3**

Pourquoi le value_counts() du "year" donne-t-il ce résultat ?

Ici, on compte le nombre de lignes de chaque année. 

In [29]:
# value_counts du gender

df_us['year'].value_counts().head(16)

2008    35084
2007    34962
2009    34715
2006    34094
2010    34082
2011    33915
2012    33757
2013    33296
2014    33266
2015    33138
2016    33028
2017    32629
2005    32551
2018    32162
2004    32049
2019    31954
Name: year, dtype: int64

In [30]:
# value_counts du gender

df_fr['year'].value_counts().head(16)

2014    13878
2012    13829
2013    13786
2011    13683
2015    13583
2016    13482
2017    13421
2010    13338
2018    13318
2019    13138
2009    12386
2008    12245
2007    12111
2006    11914
2005    11382
2004    10976
Name: year, dtype: int64

Les années, données par le value_count,sont les années où il y a eu le plus de prénoms donnés. 

**Exercice n° 1**

Donnez le prénom qui a été le plus donné lors d'une année.

In [31]:
df_fr.head()

,year,name,gender,births
0,1900,Marie,F,48713
1,1900,Jeanne,F,13981
2,1900,Marguerite,F,8058
3,1900,Germaine,F,6981
4,1900,Louise,F,6696


In [42]:
df_us.loc[df_us['births'].idxmax()]

year         1947
name        Linda
gender          F
births      99690
initial         L
terminal        A
Name: 431067, dtype: object

In [44]:
df_fr.loc[df_fr['births'].idxmax()]

year       1946
name       Jean
gender        M
births    53624
Name: 95566, dtype: object

**Exercice n° 2**

Donnez la liste des prénoms qui contiennent dans l'ordre a, e, i, o et u (US) ou bien a, e, i et o (FR).

In [47]:
df_us.loc[df_us['name'].str.contains('a.*e.*i.*o.*u'),'name'].unique()

array(['Laprecious', 'Markevious', 'Laderious', 'Jakevious',
       'Quanterious', 'Quanterrious', 'Latrevious', 'Jaterrious',
       'Jamerious', 'Jaderious', 'Marquevious', 'Laterrious', 'Jaterious',
       'Dametrious'], dtype=object)

In [49]:
df_fr.loc[df_fr['name'].str.contains('a.*e.*i.*o'),'name'].unique()

array(['Marie-Philomene', 'Marie-Victorine', 'Marie-Victoire',
       'Marie-Simone', 'Marcelino', 'Valerio', 'Valentino', 'Valeriano',
       'Saverio', 'Marie-Nicole', 'Marie-Violaine', 'Marie-Mimose',
       'Marie-Victoria', 'Marcellino', 'Maria-Conception',
       'Marie-Christophe', 'Laurentino', 'Charles-Victor',
       'Charles-Nicolas', 'Marie-Violette', 'Maelio'], dtype=object)

### Méthodes de reshaping (1)

#### pivot_table

La méthode pivot_table() prend en argument :
- values : valeurs qui doivent être agrégées selon les modalités de la colonne passée en "index" et de la colonne passée en "columns"
- index : colonne(s) dont les valeurs vont servir d'index à la table pivot
- columns : colonne(s) dont les valeurs vont servir de colonnes à la table pivot
- aggfunc : fonction d'aggrégation des values, par défaut 'mean', 'median', 'min', 'max', 'count', 'sum', 'nunique', et n'importe quelle lambda ou liste de fonctions.

On obtient NaN s'il n'y a pas d'occurence croisée.

Excel = tableau croisé dynamique.

In [50]:
# exemple
df = pd.DataFrame([{'A': 1,'B': 1, 'C': 1},
                   {'A': 1,'B': 1, 'C': 2},
                   {'A': 1,'B': 2, 'C': -1},
                   {'A': 2,'B': 1, 'C': 4},
                   {'A': 2,'B': 1, 'C': 5},
                  ])

df

,A,B,C
0,1,1,1
1,1,1,2
2,1,2,-1
3,2,1,4
4,2,1,5


Selon les arguments mis en index et colonnes => changement de comportement. 

In [52]:
# exemple
df.pivot_table(values='C',
              index='A',
              columns='B',
              aggfunc='mean')

B,1,2
A,,
1,1.5,-1.0
2,4.5,NaN


**Exercice n° 3**

Calculez une table pivot avec le nombre total de naissances par année et par genre.

In [54]:
df_us.pivot_table(values = 'births', 
                  index = 'year',
                 columns = 'gender',
                 aggfunc = 'sum')

gender,F,M
year,,
1880,90994,110490
1881,91953,100743
1882,107847,113686
1883,112319,104625
1884,129019,114442
...,...,...
2015,1781725,1913059
2016,1767902,1893471
2017,1721550,1845472


In [56]:
df_fr.pivot_table(values = 'births', 
                  index = 'year',
                 columns = 'gender',
                 aggfunc = 'sum')

gender,F,M
year,,
1900,235924,176132
1901,255718,194606
1902,259689,203009
1903,259685,206062
1904,263009,212691
...,...,...
2015,352479,373775
2016,344472,365626
2017,337400,357862


**Exercice n° 4**

Calculez une table pivot avec la diversité des prénoms (nombre de prénoms distincts) par année et par genre.

In [57]:
df_us.head()

,year,name,gender,births,initial,terminal
0,1880,Mary,F,7065,M,Y
1,1880,Anna,F,2604,A,A
2,1880,Emma,F,2003,E,A
3,1880,Elizabeth,F,1939,E,H
4,1880,Minnie,F,1746,M,E


In [59]:
df_us.pivot_table(values = 'name', 
                  index = 'year',
                 columns = 'gender',
                 aggfunc = 'nunique')

gender,F,M
year,,
1880,942,1058
1881,938,997
1882,1028,1099
1883,1054,1030
1884,1172,1125
...,...,...
2015,19099,14039
2016,18847,14181
2017,18400,14229


In [62]:
#différence avec count : ne change rien car on a une ligne par prénom (car par année et par genre 1 ligne par prénom)

df_us.pivot_table(values = 'name', 
                  index = 'year',
                 columns = 'gender',
                 aggfunc = 'count')

gender,F,M
year,,
1880,942,1058
1881,938,997
1882,1028,1099
1883,1054,1030
1884,1172,1125
...,...,...
2015,19099,14039
2016,18847,14181
2017,18400,14229


In [63]:
pd.Series([1,2,3,1]).count()

4

In [64]:
pd.Series([1,2,3,1]).nunique()

3

In [60]:
df_fr.pivot_table(values = 'name', 
                  index = 'year',
                 columns = 'gender',
                 aggfunc = 'nunique')

gender,F,M
year,,
1900,983,723
1901,1010,719
1902,1019,721
1903,1020,737
1904,1013,765
...,...,...
2015,7255,6328
2016,7163,6319
2017,7131,6290


In [68]:
pd.DataFrame.pivot_table? 

SyntaxError: invalid syntax (<ipython-input-68-33b83357d9d8>, line 1)

#### crosstab

crosstab() est une fonction de reshaping qui prend 2 colonnes d'un DataFrame en argument et produit le décompte croisé des occurrences.

On obtient 0 s'il n'y a pas d'occurence croisée.

In [70]:
# exemple
pd.crosstab(df['A'], df['B']) #ne met pas Nan comme le pivot_table mais rajoute des 0

B,1,2
A,,
1,2,1
2,2,0


In [72]:
# initial
#Calcul des lettres initiales  et lettres finales et on les passe en majuscule
df_us['initial'] = df_us['name'].str[0].str.upper()
# terminal
df_us['terminal'] = df_us['name'].str[-1].str.upper()

df_us.head()

,year,name,gender,births,initial,terminal
0,1880,Mary,F,7065,M,Y
1,1880,Anna,F,2604,A,A
2,1880,Emma,F,2003,E,A
3,1880,Elizabeth,F,1939,E,H
4,1880,Minnie,F,1746,M,E


In [36]:
# crosstab
pd.crosstab(df_us['initial'], df_us['terminal'])

terminal,A,B,C,D,E,F,G,H,I,J,...,Q,R,S,T,U,V,W,X,Y,Z
initial,,,,,,,,,,,,,,,,,,,,,
A,63865,370,717,3140,30176,504,205,11110,8124,297,...,68,6401,9744,2371,646,304,420,594,8720,753
B,12801,236,150,2986,15742,44,179,2425,2290,76,...,0,2532,1989,3801,336,52,236,272,8992,316
C,29034,418,372,2102,31177,201,789,1856,3694,63,...,0,5623,6964,2206,165,6,33,70,11772,743
D,31730,43,1070,3062,26176,43,362,3700,4471,99,...,26,2130,8156,1390,186,151,414,171,8553,377
E,22847,143,495,2465,17586,48,265,4229,1990,51,...,29,4618,2817,3395,450,13,54,169,4444,126
F,8316,0,284,1494,5597,0,190,938,468,6,...,33,1211,2091,789,57,0,1,320,2117,673
G,10542,62,18,2538,11982,196,502,995,1810,23,...,0,2227,2394,1921,18,197,0,2,4529,22
H,5921,215,19,2643,6443,95,488,2450,2165,7,...,11,2420,1765,2284,185,7,112,75,5825,120
I,10442,4,745,403,3550,14,226,1681,1027,9,...,52,864,1628,199,200,2,0,0,1695,301


In [73]:
pd.crosstab?
# keywords de crosstab = normalize  pour avoir des pourcentage

In [75]:
# crosstab mais pas encore en pourcentage
pd.crosstab(df_us['initial'], df_us['terminal'], normalize = True)

terminal,A,B,C,D,E,F,G,H,I,J,...,Q,R,S,T,U,V,W,X,Y,Z
initial,,,,,,,,,,,,,,,,,,,,,
A,0.032103,1.859856e-04,0.000360,0.001578,0.015168,2.533426e-04,0.000103,0.005585,0.004084,0.000149,...,0.000034,0.003218,0.004898,0.001192,0.000325,0.000153,2.111188e-04,0.000299,0.004383,3.785059e-04
B,0.006435,1.186287e-04,0.000075,0.001501,0.007913,2.211721e-05,0.000090,0.001219,0.001151,0.000038,...,0.000000,0.001273,0.001000,0.001911,0.000169,0.000026,1.186287e-04,0.000137,0.004520,1.588418e-04
C,0.014594,2.101135e-04,0.000187,0.001057,0.015672,1.010354e-04,0.000397,0.000933,0.001857,0.000032,...,0.000000,0.002826,0.003501,0.001109,0.000083,0.000003,1.658791e-05,0.000035,0.005917,3.734793e-04
D,0.015950,2.161455e-05,0.000538,0.001539,0.013158,2.161455e-05,0.000182,0.001860,0.002247,0.000050,...,0.000013,0.001071,0.004100,0.000699,0.000093,0.000076,2.081028e-04,0.000086,0.004299,1.895043e-04
E,0.011484,7.188093e-05,0.000249,0.001239,0.008840,2.412787e-05,0.000133,0.002126,0.001000,0.000026,...,0.000015,0.002321,0.001416,0.001707,0.000226,0.000007,2.714385e-05,0.000085,0.002234,6.333565e-05
F,0.004180,0.000000e+00,0.000143,0.000751,0.002813,0.000000e+00,0.000096,0.000471,0.000235,0.000003,...,0.000017,0.000609,0.001051,0.000397,0.000029,0.000000,5.026639e-07,0.000161,0.001064,3.382928e-04
G,0.005299,3.116516e-05,0.000009,0.001276,0.006023,9.852212e-05,0.000252,0.000500,0.000910,0.000012,...,0.000000,0.001119,0.001203,0.000966,0.000009,0.000099,0.000000e+00,0.000001,0.002277,1.105861e-05
H,0.002976,1.080727e-04,0.000010,0.001329,0.003239,4.775307e-05,0.000245,0.001232,0.001088,0.000004,...,0.000006,0.001216,0.000887,0.001148,0.000093,0.000004,5.629835e-05,0.000038,0.002928,6.031966e-05
I,0.005249,2.010655e-06,0.000374,0.000203,0.001784,7.037294e-06,0.000114,0.000845,0.000516,0.000005,...,0.000026,0.000434,0.000818,0.000100,0.000101,0.000001,0.000000e+00,0.000000,0.000852,1.513018e-04


In [76]:
# crosstab avec pourcentage en multipliant *100
pd.crosstab(df_us['initial'], df_us['terminal'], normalize = True)*100

terminal,A,B,C,D,E,F,G,H,I,J,...,Q,R,S,T,U,V,W,X,Y,Z
initial,,,,,,,,,,,,,,,,,,,,,
A,3.210263,0.018599,0.036041,0.157836,1.516838,0.025334,0.010305,0.558460,0.408364,0.014929,...,0.003418,0.321755,0.489796,0.119182,0.032472,0.015281,0.021112,0.029858,0.438323,0.037851
B,0.643460,0.011863,0.007540,0.150095,0.791293,0.002212,0.008998,0.121896,0.115110,0.003820,...,0.000000,0.127274,0.099980,0.191063,0.016890,0.002614,0.011863,0.013672,0.451995,0.015884
C,1.459434,0.021011,0.018699,0.105660,1.567155,0.010104,0.039660,0.093294,0.185684,0.003167,...,0.000000,0.282648,0.350055,0.110888,0.008294,0.000302,0.001659,0.003519,0.591736,0.037348
D,1.594952,0.002161,0.053785,0.153916,1.315773,0.002161,0.018196,0.185986,0.224741,0.004976,...,0.001307,0.107067,0.409973,0.069870,0.009350,0.007590,0.020810,0.008596,0.429928,0.018950
E,1.148436,0.007188,0.024882,0.123907,0.883985,0.002413,0.013321,0.212577,0.100030,0.002564,...,0.001458,0.232130,0.141600,0.170654,0.022620,0.000653,0.002714,0.008495,0.223384,0.006334
F,0.418015,0.000000,0.014276,0.075098,0.281341,0.000000,0.009551,0.047150,0.023525,0.000302,...,0.001659,0.060873,0.105107,0.039660,0.002865,0.000000,0.000050,0.016085,0.106414,0.033829
G,0.529908,0.003117,0.000905,0.127576,0.602292,0.009852,0.025234,0.050015,0.090982,0.001156,...,0.000000,0.111943,0.120338,0.096562,0.000905,0.009902,0.000000,0.000101,0.227656,0.001106
H,0.297627,0.010807,0.000955,0.132854,0.323866,0.004775,0.024530,0.123153,0.108827,0.000352,...,0.000553,0.121645,0.088720,0.114808,0.009299,0.000352,0.005630,0.003770,0.292802,0.006032
I,0.524882,0.000201,0.037448,0.020257,0.178446,0.000704,0.011360,0.084498,0.051624,0.000452,...,0.002614,0.043430,0.081834,0.010003,0.010053,0.000101,0.000000,0.000000,0.085202,0.015130


In [77]:
# crosstab avec une dernière ligne total en utilisant margins 
pd.crosstab(df_us['initial'], df_us['terminal'], normalize = True, margins =True )*100

terminal,A,B,C,D,E,F,G,H,I,J,...,R,S,T,U,V,W,X,Y,Z,All
initial,,,,,,,,,,,,,,,,,,,,,
A,3.210263,0.018599,0.036041,0.157836,1.516838,0.025334,0.010305,0.558460,0.408364,0.014929,...,0.321755,0.489796,0.119182,0.032472,0.015281,0.021112,0.029858,0.438323,0.037851,9.894486
B,0.643460,0.011863,0.007540,0.150095,0.791293,0.002212,0.008998,0.121896,0.115110,0.003820,...,0.127274,0.099980,0.191063,0.016890,0.002614,0.011863,0.013672,0.451995,0.015884,4.065545
C,1.459434,0.021011,0.018699,0.105660,1.567155,0.010104,0.039660,0.093294,0.185684,0.003167,...,0.282648,0.350055,0.110888,0.008294,0.000302,0.001659,0.003519,0.591736,0.037348,6.883228
D,1.594952,0.002161,0.053785,0.153916,1.315773,0.002161,0.018196,0.185986,0.224741,0.004976,...,0.107067,0.409973,0.069870,0.009350,0.007590,0.020810,0.008596,0.429928,0.018950,6.736550
E,1.148436,0.007188,0.024882,0.123907,0.883985,0.002413,0.013321,0.212577,0.100030,0.002564,...,0.232130,0.141600,0.170654,0.022620,0.000653,0.002714,0.008495,0.223384,0.006334,4.591030
F,0.418015,0.000000,0.014276,0.075098,0.281341,0.000000,0.009551,0.047150,0.023525,0.000302,...,0.060873,0.105107,0.039660,0.002865,0.000000,0.000050,0.016085,0.106414,0.033829,1.715692
G,0.529908,0.003117,0.000905,0.127576,0.602292,0.009852,0.025234,0.050015,0.090982,0.001156,...,0.111943,0.120338,0.096562,0.000905,0.009902,0.000000,0.000101,0.227656,0.001106,2.891272
H,0.297627,0.010807,0.000955,0.132854,0.323866,0.004775,0.024530,0.123153,0.108827,0.000352,...,0.121645,0.088720,0.114808,0.009299,0.000352,0.005630,0.003770,0.292802,0.006032,2.465818
I,0.524882,0.000201,0.037448,0.020257,0.178446,0.000704,0.011360,0.084498,0.051624,0.000452,...,0.043430,0.081834,0.010003,0.010053,0.000101,0.000000,0.000000,0.085202,0.015130,1.550416


In [79]:
# pivot_table 
# Si on utilise pivot_table => on aura des NaN dans le fichier, c'est pour cela que l'on met fillna
df_us.pivot_table(values='name',
                  index='initial',
                  columns='terminal',
                  aggfunc='nunique').fillna(0).astype(int)
# on dénormalise les données avec le pivot_table

terminal,A,B,C,D,E,F,G,H,I,J,...,Q,R,S,T,U,V,W,X,Y,Z
initial,,,,,,,,,,,,,,,,,,,,,
A,3551,33,28,152,1458,26,19,905,569,22,...,14,321,480,141,56,30,18,16,428,64
B,718,7,9,74,611,3,14,153,147,5,...,0,117,75,99,20,2,9,20,329,20
C,1545,22,21,57,1230,6,29,170,201,1,...,0,195,233,71,22,2,4,5,368,24
D,1737,5,28,120,1192,2,17,305,323,10,...,1,132,356,59,11,5,6,13,391,33
E,920,9,21,78,704,3,8,250,152,3,...,1,159,142,94,15,2,7,6,203,17
F,376,0,8,39,190,0,5,48,35,1,...,5,52,69,22,7,0,1,8,78,23
G,452,7,4,86,451,7,17,54,106,6,...,0,82,111,53,4,6,0,2,148,8
H,289,12,6,75,296,5,21,147,128,3,...,3,104,71,70,20,2,1,7,178,8
I,539,1,21,20,176,3,4,108,93,2,...,5,49,84,23,16,1,0,0,74,10


In [80]:
# Z x Z # les prénoms qui commencent et finissent dans Z en utilisant initial et terminal 
df_us.loc[(df_us['initial'] == 'Z') & (df_us['terminal'] == 'Z')]

,year,name,gender,births,initial,terminal
1332909,1999,Zyquez,M,5,Z,Z
1551417,2006,Zyquez,M,6,Z,Z
1583762,2007,Zyquez,M,8,Z,Z
1616535,2008,Zyquez,M,11,Z,Z
1721411,2011,Zyquez,M,8,Z,Z
1826482,2014,Zoraiz,M,5,Z,Z
1854310,2015,Zoraiz,M,9,Z,Z
1889192,2016,Zoraiz,M,7,Z,Z
1921806,2017,Zoraiz,M,7,Z,Z
1952182,2018,Zoraiz,M,9,Z,Z


In [39]:
# Z x Z
df_us.loc[df_us['name'].str.startswith('Z') & df_us['name'].str.endswith('z')]

,year,name,gender,births,initial,terminal
1332909,1999,Zyquez,M,5,Z,Z
1551417,2006,Zyquez,M,6,Z,Z
1583762,2007,Zyquez,M,8,Z,Z
1616535,2008,Zyquez,M,11,Z,Z
1721411,2011,Zyquez,M,8,Z,Z
1826482,2014,Zoraiz,M,5,Z,Z
1854310,2015,Zoraiz,M,9,Z,Z
1889192,2016,Zoraiz,M,7,Z,Z
1921806,2017,Zoraiz,M,7,Z,Z
1952182,2018,Zoraiz,M,9,Z,Z
